In [95]:
from asyncio import protocols
from datetime import datetime,timedelta,date
import pandas as pd
from pyhive import presto
import numpy as np
import os
import warnings
from scipy import stats
import datetime
from datetime import datetime
        #host='bi-trino-2.serving.data.production.internal',
        #host='bi-trino.serving.data.production.internal',
        #host='bi-presto.serving.data.production.internal',
        #host='prime-trino.serving.data.production.internal',
presto_host = "presto.processing.yoda.run"
presto_port = '80'
presto_conn = presto.connect(
    host=presto_host,
    port=presto_port,
    username='praveen.u@rapido.bike',
    protocol='http',
    catalog='hive',
)
pd.options.mode.chained_assignment = None

In [96]:
fb_data=pd.DataFrame()

for i in pd.date_range('20230424','20230504'):
    
    query="""
    
        select eventProps_city,eventProps_yyyymmdd,
        eventProps_userId,is_app_in_background,count(distinct unique_id) as total_pings
        from
        (
            Select 
            json_extract_scalar(eventProps_data, '$.is_app_in_background') as is_app_in_background,
            eventProps_city,
            eventProps_currentOrderId,
            eventProps_userId,
            eventProps_yyyymmdd,
            concat(eventProps_currentOrderId, '_',eventProps_userId) as unique_id,
            epoch
            from 
            raw.clevertap_captain_orderreceived
            where yyyymmdd >='{dt}' and yyyymmdd <='{dt}'
            and eventProps_city in ('Bangalore','Chennai','Hyderabad')
            and eventProps_userId in (select captainId from captain.captain_single_view --hive.datasets.captain_single_view 
                                                        where activationdate is not null
                                                        and lower(shift) like '%auto%'
                                                        and registeredCity in ('bangalore','chennai','hyderabad'))

        )
        
        group by 1,2,3,4
        
        """.format(dt=i.strftime('%Y%m%d'))
    
    

    data_of_app=pd.read_sql(query,presto_conn)

    #print(query)
    print(i)
    fb_data=fb_data.append(data_of_app)
    
    

2023-04-24 00:00:00
2023-04-25 00:00:00
2023-04-26 00:00:00
2023-04-27 00:00:00
2023-04-28 00:00:00
2023-04-29 00:00:00
2023-04-30 00:00:00
2023-05-01 00:00:00
2023-05-02 00:00:00
2023-05-03 00:00:00
2023-05-04 00:00:00


In [24]:
fb_data.to_csv('test_Del.csv',index=False)

In [ ]:
#get the reactivated and active captains

In [97]:
fb_data.head()

,eventProps_city,eventProps_yyyymmdd,eventProps_userId,is_app_in_background,total_pings
0,Bangalore,20230424,60e28e4e569b26fb6d713056,true,355
1,Hyderabad,20230424,5f4f6f2a551bc150b2f37639,false,11
2,Chennai,20230424,6188f795760799e44b2abb10,false,168
3,Hyderabad,20230424,5f58fd5b529dd14444d7560b,true,4
4,Bangalore,20230424,5b7e195e343935500f180f28,false,22


In [98]:
one=pd.read_csv('acctive.csv')
two=pd.read_csv('reactive.csv')

In [99]:
one['type']='active'
two['type']='reactive'

In [100]:
merged_dt=pd.concat([one,two]).reset_index(drop=True)

In [101]:
merged_dt=merged_dt.drop_duplicates(['captain_id']).reset_index(drop=True)

In [102]:
#merged_dt.merge(fb_data,left_on=['captain_id'],right_on=['eventProps_userId'])

In [103]:
test1=pd.pivot_table(data=fb_data,values=['total_pings'],index=['eventProps_city','eventProps_yyyymmdd','eventProps_userId'],columns=['is_app_in_background'],
              aggfunc='sum').reset_index()




In [104]:
test1.columns=["".join(x).strip('_') for x in test1.columns]

In [105]:
test1=test1.drop_duplicates(subset=['eventProps_yyyymmdd','eventProps_userId']).reset_index(drop=True)

In [106]:
test1.head()

,eventProps_city,eventProps_yyyymmdd,eventProps_userId,total_pingsfalse,total_pingstrue
0,Bangalore,20230423,5c7955269846eb1dabcae829,1.0,NaN
1,Bangalore,20230423,60ea9894ac6ff4e2c061532a,NaN,1.0
2,Bangalore,20230423,62570cfbabe997b4993f9f6f,1.0,NaN
3,Bangalore,20230424,573f28f79b0ffc283677073c,121.0,28.0
4,Bangalore,20230424,573f28f89b0ffc2836770c52,2.0,3.0


In [107]:
merged_dt.head()

,city,captain_id,type
0,Bangalore,61d429a57bea5e393ad267a5,active
1,Bangalore,5cb0ae2054bc7263ff35ae90,active
2,Bangalore,60ff948c2aaf7306c8b51ddf,active
3,Bangalore,5b0debfe9c89f20bdcd9371d,active
4,Bangalore,5c778f0be1692f4f9bcf608a,active


In [108]:
new_dt=merged_dt.merge(test1,left_on=['captain_id'],right_on=['eventProps_userId'])

In [109]:
f1_dt=new_dt.groupby(['city','type','eventProps_yyyymmdd'],as_index=False)[['total_pingsfalse','total_pingstrue']].sum()


In [110]:
f1_dt.to_clipboard()

In [71]:
new_dt

,city,captain_id,type,eventProps_city,eventProps_yyyymmdd,eventProps_userId,total_pingsfalse,total_pingstrue
0,Bangalore,61d429a57bea5e393ad267a5,active,Bangalore,20230505,61d429a57bea5e393ad267a5,36.0,94.0
1,Bangalore,61d429a57bea5e393ad267a5,active,Bangalore,20230506,61d429a57bea5e393ad267a5,15.0,53.0
2,Bangalore,61d429a57bea5e393ad267a5,active,Bangalore,20230507,61d429a57bea5e393ad267a5,1.0,11.0
3,Bangalore,61d429a57bea5e393ad267a5,active,Bangalore,20230508,61d429a57bea5e393ad267a5,48.0,103.0
4,Bangalore,61d429a57bea5e393ad267a5,active,Bangalore,20230509,61d429a57bea5e393ad267a5,37.0,60.0
...,...,...,...,...,...,...,...,...
1316246,Hyderabad,5fcc4f315441f88fa744f69b,reactive,Hyderabad,20230518,5fcc4f315441f88fa744f69b,5.0,8.0
1316247,Hyderabad,5fcc4f315441f88fa744f69b,reactive,Hyderabad,20230519,5fcc4f315441f88fa744f69b,2.0,5.0
1316248,Hyderabad,5fcc4f315441f88fa744f69b,reactive,Hyderabad,20230524,5fcc4f315441f88fa744f69b,7.0,4.0
1316249,Hyderabad,5fcc4f315441f88fa744f69b,reactive,Hyderabad,20230526,5fcc4f315441f88fa744f69b,34.0,20.0


In [ ]:
#with segment

In [74]:
query=""" select captain_id,yyyymmdd,city_name,performance_segment,concat(performance_segment,'_',consistency) as PC_segment
    from datasets_internal.captain_auto_pc_segments_v1
    where yyyymmdd >= '20230505' and yyyymmdd<='20230528' and city_name in ('Bangalore','Chennai','Hyderabad')
"""
#Hyderabad','Chennai','Bangalore','Mumbai','Jaipur','Delhi'
auto_caps=pd.read_sql(query,presto_conn)

In [75]:
auto_caps

,captain_id,yyyymmdd,city_name,performance_segment,PC_segment
0,5c39e61ed8a7ad6ca4fcce22,20230515,Hyderabad,MP,MP_inter
1,5c4589da9359a37c0db8cf4c,20230515,Hyderabad,LP,LP_intra
2,5c746376343a5357615e3278,20230515,Hyderabad,LP,LP_intra
3,5ca5406254bc7263ff25bc1e,20230515,Hyderabad,HP,HP_intra
4,5cc3b55554bc7263ff510de2,20230515,Hyderabad,LP,LP_inter
...,...,...,...,...,...
2773342,60e00efa09aa99d90e3ea55a,20230512,Hyderabad,LP,LP_inter
2773343,60f1ca9a8f3479793d182186,20230512,Hyderabad,HP,HP_intra
2773344,60f281202a685a4639b6b424,20230512,Hyderabad,LP,LP_inter
2773345,60fe4b9ba0ea0d71db74ac79,20230512,Hyderabad,LP,LP_intra


In [81]:
with_seg_Dt=(new_dt.merge(auto_caps[['captain_id','yyyymmdd','performance_segment']],
              left_on=['eventProps_yyyymmdd','captain_id'],
              right_on=['yyyymmdd','captain_id'],
              how='left'))

In [82]:
with_seg_Dt

,city,captain_id,type,eventProps_city,eventProps_yyyymmdd,eventProps_userId,total_pingsfalse,total_pingstrue,yyyymmdd,performance_segment
0,Bangalore,61d429a57bea5e393ad267a5,active,Bangalore,20230505,61d429a57bea5e393ad267a5,36.0,94.0,20230505,LP
1,Bangalore,61d429a57bea5e393ad267a5,active,Bangalore,20230506,61d429a57bea5e393ad267a5,15.0,53.0,20230506,LP
2,Bangalore,61d429a57bea5e393ad267a5,active,Bangalore,20230507,61d429a57bea5e393ad267a5,1.0,11.0,20230507,LP
3,Bangalore,61d429a57bea5e393ad267a5,active,Bangalore,20230508,61d429a57bea5e393ad267a5,48.0,103.0,20230508,LP
4,Bangalore,61d429a57bea5e393ad267a5,active,Bangalore,20230509,61d429a57bea5e393ad267a5,37.0,60.0,20230509,LP
...,...,...,...,...,...,...,...,...,...,...
1316252,Hyderabad,5fcc4f315441f88fa744f69b,reactive,Hyderabad,20230518,5fcc4f315441f88fa744f69b,5.0,8.0,NaN,NaN
1316253,Hyderabad,5fcc4f315441f88fa744f69b,reactive,Hyderabad,20230519,5fcc4f315441f88fa744f69b,2.0,5.0,NaN,NaN
1316254,Hyderabad,5fcc4f315441f88fa744f69b,reactive,Hyderabad,20230524,5fcc4f315441f88fa744f69b,7.0,4.0,NaN,NaN
1316255,Hyderabad,5fcc4f315441f88fa744f69b,reactive,Hyderabad,20230526,5fcc4f315441f88fa744f69b,34.0,20.0,20230526,LP


In [87]:
wiht_seg_fnl=(with_seg_Dt.groupby(['city','type','eventProps_yyyymmdd','performance_segment'],as_index=False)
[['total_pingsfalse','total_pingstrue']].sum().sort_values(by=['city','type','performance_segment']))




In [88]:
wiht_seg_fnl.to_clipboard()